In [1]:
import numpy as np
from tqdm import tqdm

In [2]:
file = "LP_transe_epochs=100embs.txt"
embedding_size = 50

f = open(f"/home/olli/gits/Better_Knowledge_Graph_Embeddings/Evaluation-Framework/{file}",'r')
lines = f.readlines()
f.close()

In [3]:
filterfile =  open("/home/olli/gits/Better_Knowledge_Graph_Embeddings/Evaluation-Framework/relevant_entities_clean.txt",'r')
relevant_entities = set([x.strip() for x in filterfile.readlines()])
f.close()

In [4]:
ents = []
vecs = []

for l in lines:
    l = l.split()
    ents.append(l[0])
    vecs.append(np.array(l[1:]))

In [5]:
unclean_mask = []
for i,(v,e)  in enumerate(zip(vecs,ents)):
    if v.shape != (embedding_size,) or e not in relevant_entities:
        unclean_mask.append(i)    

In [6]:
unclean_mask.reverse() 

In [7]:
for i in unclean_mask:
    vecs.pop(i)
    ents.pop(i)

In [8]:
vecs = np.stack(vecs)
ents = np.array(ents)

In [9]:
ents

array(['http://dbpedia.org/resource/(a)spera',
       'http://dbpedia.org/resource/...Burn,_Piano_Island,_Burn',
       'http://dbpedia.org/resource/100_Days,_100_Nights', ...,
       'http://dbpedia.org/resource/Youngstown_State_University',
       'http://dbpedia.org/resource/Your_Future_Our_Clutter',
       'http://dbpedia.org/resource/Yours_Truly,_Angry_Mob'], dtype='<U93')

In [10]:
from SPARQLWrapper import SPARQLWrapper, JSON

In [11]:
get_all_cities = """
SELECT DISTINCT ?city WHERE {
     ?city  <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Country> 

}


"""

In [12]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery(get_all_cities)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()


In [13]:
cities = set()
for result in results['results']['bindings']:
    cities.add(result['city']['value'])

In [14]:
city_boolean_mask = np.array([c in cities for c in ents])
city_ix = np.where(city_boolean_mask)[0]

In [15]:
len(cities)

10000

In [16]:
country_query = """
SELECT ?country WHERE {
<%s> <http://dbpedia.org/ontology/capital> ?country
}
"""


In [17]:
country_queries = [country_query % c for c in ents[city_boolean_mask]]

In [18]:
# save this because api slow!
countries = []
for q in tqdm(country_queries):
    sparql.setQuery(q)
    countries.append(sparql.query().convert())


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 119/119 [00:23<00:00,  5.08it/s]


In [19]:
for i,c in enumerate(countries):
    try:
        countries[i] = c['results']['bindings'][0]['country']['value']
    except:
        countries[i] = None

In [20]:
final_countries = []
final_cities = []
final_city_ix = []
final_country_ix = []

for city,country,city_index in zip(ents[city_boolean_mask],countries,city_ix):
    print(city,country)
    if country and country in ents:

        country_index = np.where(country  == ents)[0]

        final_countries.append(country)
        final_cities.append(city)
        final_country_ix.append(country_index)
        final_city_ix.append(city_index)
        

        
        
        
        

http://dbpedia.org/resource/Afghanistan http://dbpedia.org/resource/Kabul
http://dbpedia.org/resource/Albania http://dbpedia.org/resource/Tirana
http://dbpedia.org/resource/Algeria http://dbpedia.org/resource/Algiers
http://dbpedia.org/resource/Angola http://dbpedia.org/resource/Luanda
http://dbpedia.org/resource/Argentina http://dbpedia.org/resource/Buenos_Aires
http://dbpedia.org/resource/Armenia http://dbpedia.org/resource/Yerevan
http://dbpedia.org/resource/Australia http://dbpedia.org/resource/Canberra
http://dbpedia.org/resource/Austria http://dbpedia.org/resource/Vienna
http://dbpedia.org/resource/Azerbaijan http://dbpedia.org/resource/Baku
http://dbpedia.org/resource/Bahrain http://dbpedia.org/resource/Manama
http://dbpedia.org/resource/Bangladesh http://dbpedia.org/resource/Dhaka
http://dbpedia.org/resource/Belarus http://dbpedia.org/resource/Minsk
http://dbpedia.org/resource/Belgium http://dbpedia.org/resource/City_of_Brussels
http://dbpedia.org/resource/Belize http://dbpedia

In [21]:
import sklearn.manifold

In [22]:
reducer = sklearn.manifold.TSNE(learning_rate='auto',init='pca').fit_transform

In [23]:
vecs.shape

(4330, 50)

In [24]:
red = reducer(vecs)

In [25]:
import plotly.express as px

In [26]:
final_country_ix = [int(x) for x in final_country_ix]

In [27]:
final_country_ix = final_country_ix[0:20]
final_city_ix = final_city_ix[0:20]

In [28]:
import plotly.graph_objects as go

figs = []

# plot everything else

fig = go.Scatter(x=np.delete(red, final_city_ix + final_country_ix, axis=0)[:, 0],
                 y=np.delete(red, final_city_ix + final_country_ix, axis=0)[:, 1], mode="markers", showlegend=False,
                 marker=dict(color='#003f5c',size=1))
figs.append(fig)

# plot lines for city - > country
for city_ix, country_ix in zip(final_city_ix, final_country_ix):
    fig = go.Scatter(x=[red[city_ix][0], red[country_ix][0]], y=[red[city_ix][1], red[country_ix][1]], mode="lines",
                     showlegend=False, marker=dict(color='#7a5195',size=3))

    # fig.update_traces(line=(dict(color='blue')))

    figs.append(fig)

# plot city markers
fig = go.Scatter(x=red[final_city_ix][:, 0], y=red[final_city_ix][:, 1], mode="markers",
                 marker=dict(size=4, color='#ef5675'), name="countries")
figs.append(fig)

# plot country markers
fig = go.Scatter(x=red[final_country_ix][:, 0], y=red[final_country_ix][:, 1], mode="markers",
                 marker=dict(size=4, color='#ffa600'), name="cities")
figs.append(fig)

fig3 = go.Figure(data=figs)
#fig3.update_xaxes(range=(-60, 60), )

fig3.update_layout(legend=dict(title_font_family="Times New Roman",
                               font=dict(size=30)
                               ))
fig3.show(renderer='browser')
fig3.write_image(f'{file}_lines.pdf')

Opening in existing browser session.
